In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Concatenate, Dropout, BatchNormalization

In [2]:
IMAGE_DIR = "houses_dataset/houses_dataset"
TEXT_FILE = "houses_dataset/HousesInfo.txt"

In [3]:
columns = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
data = pd.read_csv(TEXT_FILE, sep=" ", names=columns)

X_table = data[["bedrooms", "bathrooms", "area", "zipcode"]].values
y = data["price"].values

scaler = MinMaxScaler()
X_table = scaler.fit_transform(X_table)

data.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226


In [4]:
def load_images(image_dir, n_samples, img_size=(128,128)):
    images = []
    for i in range(1, n_samples+1):
        house_images = []
        for room in ["frontal", "bedroom", "bathroom", "kitchen"]:
            filename = os.path.join(image_dir, f"{i}_{room}.jpg")
            if os.path.exists(filename):
                img = cv2.imread(filename)
                img = cv2.resize(img, img_size)
                house_images.append(img)
        if len(house_images) == 4:
            merged = np.concatenate(house_images, axis=1)
            images.append(merged)
    return np.array(images)

X_image = load_images(IMAGE_DIR, len(data), img_size=(64,64))
X_image = X_image / 255.0

print("Image shape:", X_image.shape)
print("Tabular shape:", X_table.shape)

Image shape: (535, 64, 256, 3)
Tabular shape: (535, 4)


In [5]:
X_image_train, X_image_test, X_table_train, X_table_test, y_train, y_test = train_test_split(X_image, X_table, y,
                                                                                             test_size=0.2, random_state=42)

In [7]:
image_input = Input(shape=(64, 256, 3))
cnn_base = ResNet50(include_top=False, input_tensor=image_input, weights="imagenet")
cnn_base.trainable = False

x = GlobalAveragePooling2D()(cnn_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

table_input = Input(shape=(4,))
y = Dense(64, activation='relu')(table_input)
y = BatchNormalization()(y)
y = Dropout(0.3)(y)
y = Dense(32, activation='relu')(y)

combined = Concatenate()([x, y])
z = Dense(128, activation='relu')(combined)
z = Dropout(0.3)(z)
z = Dense(1, activation='linear')(z)

model = Model(inputs=[image_input, table_input], outputs=z)

In [8]:
model.compile(optimizer=Adam(1e-1), loss="mse", metrics=["mae"])

In [9]:
callbacks = [EarlyStopping(monitor="val_mae", patience=10, restore_best_weights=True, verbose=1)]

In [10]:
history = model.fit(
    [X_image_train, X_table_train], y_train,
    validation_data=([X_image_test, X_table_test], y_test),
    epochs=100,
    batch_size=32,
    callbacks=callbacks)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 781ms/step - loss: 462552956928.0000 - mae: 478680.1562 - val_loss: 233815244800.0000 - val_mae: 358139.4375
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 640ms/step - loss: 329857794048.0000 - mae: 382617.7188 - val_loss: 160036274176.0000 - val_mae: 287081.2188
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 616ms/step - loss: 242234277888.0000 - mae: 299694.1562 - val_loss: 214912860160.0000 - val_mae: 420288.1250
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 532ms/step - loss: 229825085440.0000 - mae: 288873.5625 - val_loss: 156144959488.0000 - val_mae: 349855.0625
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 535ms/step - loss: 229340872704.0000 - mae: 292526.9062 - val_loss: 129218404352.0000 - val_mae: 312247.0312
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 523ms/step - loss: 223609405440.0000 - mae: 277513.7500 - val_loss: 146647154688.0000 - val_mae: 337379.0000
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 522ms/step - loss: 205772865536.0000 - mae: 285112.

In [11]:
loss, mae = model.evaluate([X_image_test, X_table_test], y_test)

print(f"Test Loss: {loss:.2f}")
print(f"Test MAE: {mae:.2f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 309ms/step - loss: 99823534080.0000 - mae: 214150.5781 
Test Loss: 99823534080.00
Test MAE: 214150.58
